In [113]:
import pandas as pd
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn import svm

In [39]:
data = pd.read_csv('train_data.csv')

In [40]:
data.head()

,f1,f2,f3,y
0,G40,V1507,w80037 w145228 w21671 w72264 w73158 w108141 w5...,2
1,G6,V2538,w20895 w100716 w95915 w83837 w121585 w30607 w2...,7
2,G103,V601,w57656 w141678 w59483 w59680 w42932 w100072 w9...,1
3,G35,V2322,w120952 w115518 w144391 w110805 w116503 w11645...,1
4,G16,V943,w84661 w65916 w42618 w88730 w131742 w64558 w62...,7


In [41]:
data.shape

(2656, 4)

In [42]:
data.dtypes

f1     object
 f2    object
 f3    object
 y      int64
dtype: object

In [43]:
data.describe(include=['object'])

,f1,f2,f3
count,2656,2656,2656
unique,244,2389,1614
top,G0,V0,w116338 w110805 w138213 w74239 w110805 w145552...
freq,217,83,44


In [44]:
data.columns

Index(['f1', ' f2', ' f3', ' y'], dtype='object')

In [45]:
data.rename(columns={'f1':'Feature_1', ' f2':'Feature_2',' f3':'Feature_3', ' y':'Label'}, inplace=True)
data.columns

Index(['Feature_1', 'Feature_2', 'Feature_3', 'Label'], dtype='object')

In [46]:
data.head()

,Feature_1,Feature_2,Feature_3,Label
0,G40,V1507,w80037 w145228 w21671 w72264 w73158 w108141 w5...,2
1,G6,V2538,w20895 w100716 w95915 w83837 w121585 w30607 w2...,7
2,G103,V601,w57656 w141678 w59483 w59680 w42932 w100072 w9...,1
3,G35,V2322,w120952 w115518 w144391 w110805 w116503 w11645...,1
4,G16,V943,w84661 w65916 w42618 w88730 w131742 w64558 w62...,7


In [47]:
labels = data.drop(['Feature_1', 'Feature_2', 'Feature_3'], axis=1, inplace=False)
labels.head()

,Label
0,2
1,7
2,1
3,1
4,7


In [48]:
training_data = data.drop('Label', axis=1, inplace=False)


In [49]:
training_data.head()

,Feature_1,Feature_2,Feature_3
0,G40,V1507,w80037 w145228 w21671 w72264 w73158 w108141 w5...
1,G6,V2538,w20895 w100716 w95915 w83837 w121585 w30607 w2...
2,G103,V601,w57656 w141678 w59483 w59680 w42932 w100072 w9...
3,G35,V2322,w120952 w115518 w144391 w110805 w116503 w11645...
4,G16,V943,w84661 w65916 w42618 w88730 w131742 w64558 w62...


In [50]:
Allfeature3 = {}
count = 1

for feature in training_data['Feature_3']:
    #print(feature)
    #count = count + 1
    #print(count)
    for single_feature in feature.split():
        if single_feature not in Allfeature3.keys():
            Allfeature3[single_feature] = 1
        else:
            Allfeature3[single_feature] = Allfeature3[single_feature] + 1
    
#print(count)

In [51]:
Allfeature3

{'w80037': 5000,
 'w145228': 180,
 'w21671': 40587,
 'w72264': 326,
 'w73158': 1335,
 'w108141': 21151,
 'w52677': 482,
 'w143548': 55306,
 'w115098': 10429,
 'w103020': 22593,
 'w36493': 17060,
 'w78337': 19937,
 'w30594': 3674,
 'w38997': 102446,
 'w92858': 411,
 'w107504': 2655,
 'w42393': 3195,
 'w89466': 17183,
 'w33752': 18552,
 'w115470': 5321,
 'w32646': 5035,
 'w24589': 18,
 'w57567': 85155,
 'w23584': 84974,
 'w6770': 5413,
 'w102259': 1369,
 'w87871': 8349,
 'w21663': 2071,
 'w31896': 50686,
 'w78901': 1801,
 'w120952': 20660,
 'w108043': 8,
 'w116307': 621,
 'w49659': 8,
 'w63072': 24847,
 'w115518': 68378,
 'w44246': 1574,
 'w68851': 59,
 'w131358': 58,
 'w131332': 3593,
 'w21665': 12117,
 'w67541': 427,
 'w141678': 14393,
 'w59504': 8557,
 'w104330': 13506,
 'w43550': 598,
 'w88952': 19545,
 'w97772': 345,
 'w42618': 11407,
 'w24470': 12239,
 'w78607': 6222,
 'w78251': 241,
 'w36883': 4081,
 'w30841': 976,
 'w122235': 5053,
 'w36375': 79737,
 'w145565': 1071,
 'w133178': 

In [52]:
len(Allfeature3)

137690

In [53]:
len(set(Allfeature3.keys()))

137690

In [54]:
import operator
#x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_Allfeature3 = sorted(Allfeature3.items(), key=operator.itemgetter(1) , reverse=True)
sorted_Allfeature3

[('w97932', 188476),
 ('w1892', 172403),
 ('w39052', 148918),
 ('w6659', 123653),
 ('w9621', 108715),
 ('w38997', 102446),
 ('w57567', 85155),
 ('w23584', 84974),
 ('w97864', 84564),
 ('w60641', 84344),
 ('w13446', 83600),
 ('w34675', 82569),
 ('w0', 80664),
 ('w60757', 79930),
 ('w36375', 79737),
 ('w108233', 68532),
 ('w115518', 68378),
 ('w11798', 67837),
 ('w58581', 62291),
 ('w21755', 61399),
 ('w97787', 58233),
 ('w144014', 57861),
 ('w143548', 55306),
 ('w147562', 53531),
 ('w24435', 53527),
 ('w83837', 52693),
 ('w52110', 52501),
 ('w146152', 50983),
 ('w31896', 50686),
 ('w65916', 49994),
 ('w33593', 49298),
 ('w105781', 48789),
 ('w48117', 48612),
 ('w25094', 47193),
 ('w29957', 45699),
 ('w150055', 45407),
 ('w51830', 45071),
 ('w1893', 44803),
 ('w15019', 43004),
 ('w143631', 42318),
 ('w143936', 42223),
 ('w97821', 41896),
 ('w136658', 41803),
 ('w55084', 41582),
 ('w21671', 40587),
 ('w92120', 39724),
 ('w16298', 38251),
 ('w122709', 37107),
 ('w129838', 35278),
 ('w12016

In [55]:
def featurize(data):
    
    repeated_words= {}
    #count = 1
    
    for feature in data:
        #count = count + 1
        if feature not in repeated_words.keys():
            repeated_words[feature] = 1
        else:
            repeated_words[feature] = repeated_words[feature] + 1
    
    
    return(sorted(repeated_words.items(), key=lambda x: x[0]))

In [56]:
vocabulary = {}
feature_freq = {}
features =  []


for row in training_data['Feature_3']:
    data = row.split()
    featurize_data = featurize(data)
    
    for single_feature in featurize_data:
        if single_feature[0] not in feature_freq.keys():
            feature_freq[single_feature[0]] = 1
        else:
            feature_freq[single_feature[0]] = feature_freq[single_feature[0]] + 1
            
        if  single_feature[0] not in vocabulary.keys():
            vocabulary.setdefault(single_feature[0], None)  
            
    features.append(featurize_data)
    

In [57]:
vocabulary

{'w0': None,
 'w100065': None,
 'w100072': None,
 'w100125': None,
 'w100131': None,
 'w100133': None,
 'w100159': None,
 'w100167': None,
 'w100186': None,
 'w100214': None,
 'w100419': None,
 'w100591': None,
 'w10065': None,
 'w10066': None,
 'w100714': None,
 'w100716': None,
 'w100754': None,
 'w10085': None,
 'w100872': None,
 'w100951': None,
 'w101181': None,
 'w101223': None,
 'w101408': None,
 'w101625': None,
 'w102134': None,
 'w102237': None,
 'w102256': None,
 'w102259': None,
 'w102370': None,
 'w102372': None,
 'w102394': None,
 'w102432': None,
 'w102447': None,
 'w10253': None,
 'w102907': None,
 'w102931': None,
 'w102967': None,
 'w102972': None,
 'w102997': None,
 'w103001': None,
 'w103020': None,
 'w103033': None,
 'w103313': None,
 'w103349': None,
 'w103352': None,
 'w103365': None,
 'w103366': None,
 'w103380': None,
 'w103402': None,
 'w103404': None,
 'w103463': None,
 'w103466': None,
 'w103474': None,
 'w103482': None,
 'w103866': None,
 'w104044': None,
 

In [58]:
len(vocabulary)

137690

In [59]:
features[0]

[('w0', 10),
 ('w100065', 3),
 ('w100072', 1),
 ('w100125', 1),
 ('w100131', 3),
 ('w100133', 1),
 ('w100159', 3),
 ('w100167', 1),
 ('w100186', 1),
 ('w100214', 1),
 ('w100419', 3),
 ('w100591', 1),
 ('w10065', 2),
 ('w10066', 3),
 ('w100714', 2),
 ('w100716', 5),
 ('w100754', 5),
 ('w10085', 1),
 ('w100872', 1),
 ('w100951', 1),
 ('w101181', 1),
 ('w101223', 2),
 ('w101408', 16),
 ('w101625', 10),
 ('w102134', 1),
 ('w102237', 23),
 ('w102256', 12),
 ('w102259', 1),
 ('w102370', 4),
 ('w102372', 2),
 ('w102394', 2),
 ('w102432', 1),
 ('w102447', 2),
 ('w10253', 2),
 ('w102907', 1),
 ('w102931', 1),
 ('w102967', 3),
 ('w102972', 3),
 ('w102997', 1),
 ('w103001', 1),
 ('w103020', 23),
 ('w103033', 3),
 ('w103313', 1),
 ('w103349', 1),
 ('w103352', 1),
 ('w103365', 2),
 ('w103366', 17),
 ('w103380', 1),
 ('w103402', 1),
 ('w103404', 11),
 ('w103463', 1),
 ('w103466', 3),
 ('w103474', 2),
 ('w103482', 1),
 ('w103866', 1),
 ('w104044', 1),
 ('w104330', 20),
 ('w104407', 13),
 ('w10443', 1

In [60]:
feature_freq

{'w0': 2525,
 'w100065': 1031,
 'w100072': 940,
 'w100125': 7,
 'w100131': 1183,
 'w100133': 1007,
 'w100159': 1990,
 'w100167': 731,
 'w100186': 12,
 'w100214': 854,
 'w100419': 727,
 'w100591': 41,
 'w10065': 2035,
 'w10066': 177,
 'w100714': 52,
 'w100716': 1952,
 'w100754': 1754,
 'w10085': 67,
 'w100872': 1203,
 'w100951': 122,
 'w101181': 18,
 'w101223': 1163,
 'w101408': 1190,
 'w101625': 1884,
 'w102134': 401,
 'w102237': 1955,
 'w102256': 1111,
 'w102259': 733,
 'w102370': 1598,
 'w102372': 1254,
 'w102394': 874,
 'w102432': 1691,
 'w102447': 608,
 'w10253': 1968,
 'w102907': 1187,
 'w102931': 9,
 'w102967': 1566,
 'w102972': 556,
 'w102997': 18,
 'w103001': 787,
 'w103020': 2367,
 'w103033': 1204,
 'w103313': 436,
 'w103349': 1596,
 'w103352': 1525,
 'w103365': 586,
 'w103366': 2478,
 'w103380': 24,
 'w103402': 855,
 'w103404': 2183,
 'w103463': 160,
 'w103466': 1774,
 'w103474': 1255,
 'w103482': 1156,
 'w103866': 1410,
 'w104044': 268,
 'w104330': 1516,
 'w104407': 2570,
 '

In [61]:
feature_freq.items()

dict_items([('w0', 2525), ('w100065', 1031), ('w100072', 940), ('w100125', 7), ('w100131', 1183), ('w100133', 1007), ('w100159', 1990), ('w100167', 731), ('w100186', 12), ('w100214', 854), ('w100419', 727), ('w100591', 41), ('w10065', 2035), ('w10066', 177), ('w100714', 52), ('w100716', 1952), ('w100754', 1754), ('w10085', 67), ('w100872', 1203), ('w100951', 122), ('w101181', 18), ('w101223', 1163), ('w101408', 1190), ('w101625', 1884), ('w102134', 401), ('w102237', 1955), ('w102256', 1111), ('w102259', 733), ('w102370', 1598), ('w102372', 1254), ('w102394', 874), ('w102432', 1691), ('w102447', 608), ('w10253', 1968), ('w102907', 1187), ('w102931', 9), ('w102967', 1566), ('w102972', 556), ('w102997', 18), ('w103001', 787), ('w103020', 2367), ('w103033', 1204), ('w103313', 436), ('w103349', 1596), ('w103352', 1525), ('w103365', 586), ('w103366', 2478), ('w103380', 24), ('w103402', 855), ('w103404', 2183), ('w103463', 160), ('w103466', 1774), ('w103474', 1255), ('w103482', 1156), ('w1038

In [62]:
# sort vocab according to features (alphabetical order)
sort_feature_freq = sorted(feature_freq.items(), key =lambda x: x[1], reverse=True)
sort_feature_freq

[('w6659', 2644),
 ('w1892', 2642),
 ('w13446', 2633),
 ('w9621', 2633),
 ('w24435', 2630),
 ('w11798', 2624),
 ('w122709', 2621),
 ('w97932', 2605),
 ('w15019', 2598),
 ('w25094', 2591),
 ('w16298', 2590),
 ('w115518', 2588),
 ('w39052', 2587),
 ('w146152', 2586),
 ('w1893', 2586),
 ('w48117', 2575),
 ('w74513', 2575),
 ('w92120', 2574),
 ('w143936', 2571),
 ('w104407', 2570),
 ('w97864', 2566),
 ('w144014', 2564),
 ('w129838', 2561),
 ('w34675', 2556),
 ('w38997', 2548),
 ('w17291', 2547),
 ('w149652', 2531),
 ('w62077', 2531),
 ('w0', 2525),
 ('w146123', 2525),
 ('w18147', 2521),
 ('w58581', 2520),
 ('w2948', 2517),
 ('w6660', 2511),
 ('w114347', 2499),
 ('w65916', 2497),
 ('w49747', 2495),
 ('w134042', 2489),
 ('w24506', 2486),
 ('w4189', 2482),
 ('w122136', 2479),
 ('w103366', 2478),
 ('w76713', 2478),
 ('w140200', 2476),
 ('w21755', 2474),
 ('w52110', 2469),
 ('w78337', 2468),
 ('w22002', 2459),
 ('w130440', 2452),
 ('w3814', 2452),
 ('w2450', 2450),
 ('w133995', 2444),
 ('w36375

In [63]:
# sort vocab according to features (alphabetical order)
vacab_list = sorted(feature_freq.keys(), key =lambda x: x,reverse=False)
        
for colIndex,term in enumerate(vacab_list) :
    #print('colIndex = %d, term = %s'%(colIndex,term))
    vocabulary[term] = colIndex

In [64]:
vacab_list

['w0',
 'w1',
 'w10',
 'w100',
 'w1000',
 'w10000',
 'w100000',
 'w100002',
 'w100003',
 'w100004',
 'w100005',
 'w100006',
 'w100007',
 'w100008',
 'w100009',
 'w10001',
 'w100010',
 'w100011',
 'w100012',
 'w100014',
 'w100015',
 'w100016',
 'w100017',
 'w100018',
 'w100019',
 'w10002',
 'w100020',
 'w100021',
 'w100022',
 'w100023',
 'w100024',
 'w100025',
 'w100027',
 'w100028',
 'w100029',
 'w10003',
 'w100030',
 'w100033',
 'w100034',
 'w100035',
 'w100036',
 'w100037',
 'w100038',
 'w100039',
 'w10004',
 'w100040',
 'w100042',
 'w100044',
 'w100045',
 'w100046',
 'w100047',
 'w100048',
 'w100049',
 'w10005',
 'w100050',
 'w100053',
 'w100054',
 'w100055',
 'w100056',
 'w100057',
 'w100058',
 'w100059',
 'w10006',
 'w100060',
 'w100061',
 'w100062',
 'w100063',
 'w100064',
 'w100065',
 'w100066',
 'w100067',
 'w100068',
 'w100069',
 'w10007',
 'w100070',
 'w100071',
 'w100072',
 'w100073',
 'w100074',
 'w100077',
 'w100078',
 'w100079',
 'w10008',
 'w100080',
 'w100081',
 'w10008

In [65]:
vocabulary['w0']

0

In [66]:
# build a csr_matrix    
row = []
col = []
data = []  

for colID,feat_list in enumerate(features) :
    for term in feat_list:
        
        #if (term[1] > 10):
            #print(term[0], term[1])
        
        if (feature_freq[term[0]] > 5): # (zero values are not stored)
                                                  
            row.append(colID)
            col.append(vocabulary[term[0]])
            data.append(term[1])
     




In [67]:
print('row =',row[0])
print('col =',col[0])
print('data=',data[0])

#for i in data:
    #print (i)

row = 0
col = 0
data= 10


In [68]:
print('row =',row[1])
print('col =',col[1])
print('data=',data[1])

row = 0
col = 68
data= 3


In [69]:
X = csr_matrix((data, (row, col)), shape=(len(features), len(vocabulary)), dtype=np.int64)

In [70]:
X.toarray()

array([[10,  0,  0, ...,  0,  0,  0],
       [ 2,  0,  0, ...,  0,  0,  0],
       [17,  0,  0, ...,  0,  0,  0],
       ..., 
       [52,  0,  0, ...,  0,  0,  0],
       [ 3,  0,  0, ...,  0,  0,  0],
       [48,  0,  0, ...,  0,  0,  0]], dtype=int64)

In [71]:
X.toarray().shape

(2656, 137690)

In [72]:
type(X)

scipy.sparse.csr.csr_matrix

In [97]:
def cross_validation_accuracy(clf, X_test, Labels, k):
    """
    Compute the average testing accuracy over k folds of cross-validation. You
    can use sklearn's KFold class here (no random seed, and no shuffling
    needed).

    Params:
      clf......A LogisticRegression classifier.
      X........A csr_matrix of features.
      labels...The true labels for each instance in X
      k........The number of cross-validation folds.

    Returns:
      The average testing accuracy of the classifier
      over each fold of cross-validation.
    """
    ###TODO

    cv = KFold(n=len(Labels),n_folds=k)
    accuracies = []

    print(cv)
     
    for train_indices, test_indices in cv:
        
        #print(train_indices, test_indices)
        #print(X_test[train_indices])
        print(train_indices)
        print(Labels)
        print(type(Labels))
       
        clf.fit(X_test[train_indices], Labels[train_indices])
        predicted = clf.predict(X_test[test_indices])
        
        acc = accuracy_score(Labels[test_indices], predicted)
        print(acc)
        accuracies.append(acc)
    
    #print('accuracies = ',accuracies) 
    #avg = np.mean(accuracies,dtype=np.float64)
    return(np.mean(accuracies,dtype=np.float64))   

In [109]:
clf = LogisticRegression()
avg_acc = cross_validation_accuracy(clf, X, l, k=5)

sklearn.cross_validation.KFold(n=2656, n_folds=5, shuffle=False, random_state=None)
[ 532  533  534 ..., 2653 2654 2655]
[[2]
 [7]
 [1]
 ..., 
 [7]
 [1]
 [2]]
<class 'numpy.ndarray'>


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


NameError: name 'accuracy_score' is not defined

In [120]:
#label_list = labels
labels_matrix = labels.as_matrix()
labels_matrix[500]

array([7], dtype=int64)

In [111]:
#label_list[ 532,  533,  534 ]
#store lables into array
labels_array = labels.values

#check
labels_array

array([[2],
       [7],
       [1],
       ..., 
       [7],
       [1],
       [2]], dtype=int64)

In [117]:
X_train, X_test, y_train, y_test = train_test_split( X, labels_matrix, test_size=0.2, random_state=0)

clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)                           


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.59774436090225569

In [118]:
X_train.shape, y_train.shape

((2124, 137690), (2124, 1))

In [119]:
X_test.shape, y_test.shape

((532, 137690), (532, 1))